In [1]:
# from confluent_kafka import Producer
from cassandra.cluster import Cluster
from cassandra import query
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.cqlengine import connection
from cassandra.cqlengine import management
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
import pandas as pd
import datetime
import numpy as np
# import faust
import os
import uuid
import pandas as pd

In [141]:
CASSANDRA_KEYSPACE = 'presidents'
CASSANDRA_CLUSTER_NAME = 'cluster1'

cluster = Cluster()
session = cluster.connect()
connection.register_connection(CASSANDRA_CLUSTER_NAME, session=session, default=True)

os.environ['CQLENG_ALLOW_SCHEMA_MANAGEMENT'] = '1'

management.drop_keyspace(CASSANDRA_KEYSPACE)
management.create_keyspace_simple(name=CASSANDRA_KEYSPACE, replication_factor=3)

columns.DateTime.truncate_microseconds = False


class GameClicks(Model):
    __table_name__ = 'game_clicks'
    __keyspace__ = CASSANDRA_KEYSPACE
    game_id = columns.UUID(primary_key=True, partition_key=True, required=True)
    user_id = columns.UUID(primary_key=True, partition_key=True, required=True)
    action = columns.Text(primary_key=True, required=True, clustering_order='ASC')
    timestamps = columns.List(columns.DateTime, required=True)


management.drop_table
management.sync_table(GameClicks)

In [142]:
game_id1, game_id2 = uuid.uuid4(), uuid.uuid4()
user_id1, user_id2 = uuid.uuid4(), uuid.uuid4()

In [143]:
GameClicks.objects(
    game_id=game_id2,
    user_id=user_id1,
    action='play'
).update(
    timestamps__append=[datetime.datetime.utcnow()]
)

In [144]:
def game_clicks_to_pandas(raw_rows, query):
    row_labels, row_timestamps = (list() for _ in range(2))
    for raw_row in raw_rows:
        row_vals = list(raw_row.values())
        labels, timestamps = row_vals[:3], row_vals[3]
        for timestamp in timestamps:
            row_labels.append(labels)
            row_timestamps.append(timestamp)
    index = pd.MultiIndex.from_tuples(
        row_labels,
        names=list(raw_row)[:3]
    )
    return pd.DataFrame(
        row_timestamps,
        index=index,
        columns=['timestamp']
    )

In [145]:
query = '''
    select *
    from presidents.game_clicks
'''
game_clicks_to_pandas(session.execute(query), query)

,,,timestamp
game_id,user_id,action,
ac99690d-10be-4c0e-8f2c-da6a21a226a0,209a8c20-9e63-420f-9014-ac991dbc70f2,play,2019-06-01 06:30:10.559


In [80]:
row_labels

[[UUID('30514fdf-80ab-4d2f-8cae-b4d45da21c3e'),
  UUID('e6a0eff9-599c-44c3-a759-24eeac984436'),
  'pass'],
 [UUID('30514fdf-80ab-4d2f-8cae-b4d45da21c3e'),
  UUID('e6a0eff9-599c-44c3-a759-24eeac984436'),
  'play'],
 [UUID('41b3fa6e-b7ef-48f2-bfa9-9e7ca0e38fd1'),
  UUID('e6a0eff9-599c-44c3-a759-24eeac984436'),
  'pass'],
 [UUID('41b3fa6e-b7ef-48f2-bfa9-9e7ca0e38fd1'),
  UUID('e6a0eff9-599c-44c3-a759-24eeac984436'),
  'pass']]

In [81]:
row_timestamps

[datetime.datetime(2019, 6, 1, 5, 14, 9, 63000),
 datetime.datetime(2019, 6, 1, 5, 11, 37, 567000),
 datetime.datetime(2019, 6, 1, 5, 14, 11, 810000),
 datetime.datetime(2019, 6, 1, 5, 21, 14, 68000)]

In [64]:
game_ids, user_ids, actions = (list() for _ in range(3))

In [82]:
pd.DataFrame(row_timestamps, index=pd.MultiIndex.from_tuples(row_labels))

0
30514fdf-80ab-4d2f-8cae-b4d45da21c3e e6a0eff9-599c-44c3-a759-24eeac984436 pass 2019-06-01 05:14:09.063
                                                                          play 2019-06-01 05:11:37.567
41b3fa6e-b7ef-48f2-bfa9-9e7ca0e38fd1 e6a0eff9-599c-44c3-a759-24eeac984436 pass 2019-06-01 05:14:11.810
                                                                          pass 2019-06-01 05:21:14.068

In [23]:
producer = Producer({'bootstrap.servers': 'localhost:9092'})

In [254]:
data = {
    'game_id': '1',
    'username': 'avi',
    'action': 'play',
    'timestamp': timestamp()
}

producer.produce('click',
                 json.dumps(data)
)

In [276]:
CASSANDRA_CONTACT_POINTS = ['172.17.0.2']
CASSANDRA_KEYSPACE = 'presidents'
CASSANDRA_CLUSTER_NAME = 'cluster1'

In [278]:
cluster = Cluster(CASSANDRA_CONTACT_POINTS)
session = cluster.connect()
connection.register_connection(CASSANDRA_CLUSTER_NAME, session=session, default=True);

In [281]:
os.environ['CQLENG_ALLOW_SCHEMA_MANAGEMENT'] = '1'

management.drop_keyspace(CASSANDRA_KEYSPACE)
management.create_keyspace_simple(name=CASSANDRA_KEYSPACE, replication_factor=3)

In [298]:
DateTime.truncate_microseconds = False

class GameClick(Model):
    __table_name__ = 'game_click'
    __keyspace__ = 'presidents'
    game_id = columns.Text(primary_key=True, partition_key=True, required=True)
    username = columns.Text(primary_key=True, partition_key=True, required=True)
    action = columns.Text(primary_key=True, required=True, clustering_order='ASC')
    timestamps = columns.List(columns.DateTime, required=True)

management.drop_table
management.sync_table(GameClick)

In [299]:
session.row_factory = query.tuple_factory
rows = session.execute(
    '''
    select *
    from system_schema.columns
    where keyspace_name = 'presidents'
        and table_name = 'game_click'
    '''
)

In [300]:
for row in rows:
    print(row)

('presidents', 'game_click', 'action', 'ASC', b'action', 'clustering', 0, 'text')
('presidents', 'game_click', 'game_id', 'NONE', b'game_id', 'partition_key', 0, 'text')
('presidents', 'game_click', 'timestamps', 'NONE', b'timestamps', 'regular', -1, 'list<timestamp>')
('presidents', 'game_click', 'username', 'NONE', b'username', 'partition_key', 1, 'text')


In [35]:
from db_conn import session
from click_app import app, GameClick
import datetime
import numpy as np
import pandas as pd

In [36]:
def pandas_factory(colnames, rows):
    rows = np.array([list(row) for row in rows], dtype=object)
    return pd.DataFrame(np.append(rows[:, :2], np.array(list(map(dict, rows[:, 2]))).reshape(len(rows), 1), 1), columns=colnames)

In [37]:
session.row_factory = pandas_factory # query.tuple_factory
session.default_fetch_size = None

In [39]:
await app.send('click',
               value=GameClick(
                   game_id='1',
                   username='avi',
                   timestamp=datetime.datetime.utcnow(),
                   action='play'
               )
              )

<Future pending cb=[Topic._on_published(message=<FutureMessage pending>, state={}, producer=<Producer: running >)()]>

In [11]:
session.execute(
    '''
    select *
    from presidents.game_clicks
    '''
)._current_rows

ERROR:cassandra.cluster:Unexpected exception while handling result in ResponseFuture:
Traceback (most recent call last):
  File "cassandra/cluster.py", line 3860, in cassandra.cluster.ResponseFuture._set_result
  File "<ipython-input-2-0708f6120f81>", line 3, in pandas_factory
    return pd.DataFrame(np.append(rows[:, :2], np.array(list(map(dict, rows[:, 2]))).reshape(len(rows), 1), 1), columns=colnames)
IndexError: too many indices for array


IndexError: too many indices for array

In [10]:
session.prepare(
    '''
    update presidents.game_clicks
    set timestamps = timestamps + [?]
    where game_id=?
        and username=?
        and action=?
    '''
)

ERROR:cassandra.cluster:Error preparing query:
Traceback (most recent call last):
  File "cassandra/cluster.py", line 2463, in cassandra.cluster.Session.prepare
  File "cassandra/cluster.py", line 4131, in cassandra.cluster.ResponseFuture.result
cassandra.InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid list literal for timestamps: bind variables are not supported inside collection literals"


InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid list literal for timestamps: bind variables are not supported inside collection literals"

In [8]:
session.execute(
    '''
    update presidents.game_click
    set timestamps = timestamps + [?]
    where game_id=?
        and username=?
        and action=?
    '''
)

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table game_click"

In [291]:
session.execute(
    f'''
    update presidents.click
    set action_timestamps_dict['3'] = [1]
    where game_id='1'
        and username='avi'
    '''
)

In [292]:
session.execute(
    '''
    select *
    from presidents.game_clicks
    '''
)._current_rows

,game_id,username,action_timestamps_dict
0,1,avi,"{'1': [2019-05-27 00:40:06.066000], '3': [1970..."
1,1,evilavi,{'2': [2019-05-27 00:40:06.067000]}


In [56]:
for instance in Click.objects().all():
    print(instance.action_timestamps_dict)

{'1': [datetime.datetime(2019, 5, 26, 18, 8, 35, 934000)]}


In [255]:
from click_app import app, Click, print_click

In [261]:
await app.send('click', value=Click(game_id='1', username='avi', action='play', timestamp=datetime.now()))

<Future pending cb=[Topic._on_published(message=<FutureMessage pending>, state={}, producer=<Producer: running >)()]>

In [7]:
print_click.send(

<coroutine object Agent.send at 0x7fd03449d7c8>

In [10]:
async def click():
    await print_click.send(
        value=Click(game_id='1', username='avi', action='play', timestamp=datetime.datetime.now())
    )

In [20]:
click()

<coroutine object click at 0x7fd034263548>

In [21]:
@app.timer(interval=1.0)
async def example_clicker(app):
    await print_click.send(
        value=Click(game_id='1', username='avi', action='play', timestamp=datetime.datetime.now())
    )

In [22]:
example_clicker()

<coroutine object example_clicker at 0x7fd034263648>

In [19]:
import faust
import datetime

class Click(faust.Record):
    game_id: str
    username: str
    action: str
    timestamp: datetime.datetime
        
app = faust.App('click-app', broker='kafka://localhost:9092')
topic = app.topic('click', value_type=Click)

@app.agent(topic)
async def print_click(clicks):
    async for click in clicks:
        print(
            f'''
            game_id: {click.game_id}
            username: {click.username}
            action: {click.action}
            timestamp: {click.timestamp}
            '''
        )

In [33]:
import faust

In [17]:
%timeit faust.utils.iso8601.parse('2019-05-27T21:15:27.890826')

229 ns ± 1.61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
datetime.datetime.now().isoformat()

'2019-05-27T21:28:38.226721'

In [40]:
faust.utils.iso8601.parse('2019-05-27T21:15:27.8908')

datetime.datetime(2019, 5, 27, 21, 15, 27, 890800)